In [66]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from prophet import Prophet
import warnings
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import ParameterGrid
from prophet.plot import plot_plotly, plot_components_plotly

warnings.filterwarnings('ignore')

### 01 - Prepare Dataset

In [2]:
shelter_df = pd.read_csv('DHS_weekly.csv')

column_dict = {'Date': 'ds',
               'Total Individuals in Shelter': 'y'}

shelter_df.rename(columns=column_dict, inplace=True)
shelter_df['ds'] = pd.to_datetime(shelter_df['ds'], format='%m/%d/%Y')

In [ ]:
shelter_df.tail()

### Exploratory Data Analysis

##### Time-Series Visualization

In [ ]:
sns.lineplot(shelter_df, x='ds', y='y')

2021-01-03 seems to be wrong. We'll remove it from our analysis.

In [5]:
shelter_df = shelter_df[:-1]

In [ ]:
sns.lineplot(shelter_df, x='ds', y='y')

#### Distribution of individuals in shelter

In [ ]:
sns.histplot(shelter_df['y'], kde=True);

#### Autocorrelation and Partial Autocorrelation

In [ ]:
plot_acf(shelter_df.set_index('ds')[['y']]);
plot_pacf(shelter_df.set_index('ds')[['y']]);


The correlation to the day before is statiscally significant. Not with other lags.

#### Distribution of temperature

In [ ]:
sns.histplot(shelter_df['Temperature'], kde=True);

#### Relationship between temperature and people seeking shelter

In [ ]:
sns.regplot(shelter_df, x='Temperature', y='y')

There does not seem to be a correlation between the variables. However, removing the first and last months of data reveals an underlying correlation between them.

In [ ]:
sns.regplot(shelter_df[30:-30], x='Temperature', y='y');

In [ ]:
pearsonr(shelter_df['Temperature'], shelter_df['y'])

In [ ]:
spearmanr(shelter_df['Temperature'], shelter_df['y'])

There is a statiscally significant negative correlation, though weak.

#### Seasonal Decomposition

In [15]:
seasonal_decomposition = seasonal_decompose(shelter_df.set_index('ds')['y'])

In [ ]:
seasonal_decomposition.seasonal[:110].plot();

### 2 - Build the model

#### Prepare holidays df

In [ ]:
holidays = pd.DataFrame({
    'holiday': 'Easter',
    'ds': shelter_df['ds'][shelter_df['Easter'] == 1],
    'lower_window': 0,
    'upper_window': 1,
})

for holiday in ['Thanksgiving', 'Christmas']:
    temp = pd.DataFrame({
        'holiday': holiday,
        'ds': shelter_df['ds'][shelter_df[holiday] == 1],
        'lower_window': 0,
        'upper_window': 1,
    })
    holidays = pd.concat([holidays, temp])

holidays

#### Split dataset

In [15]:
df_train, df_test = shelter_df[:-13], shelter_df[-13:]

In [ ]:
m = Prophet(holidays=holidays)
m.add_regressor('Temperature')

m.fit(df_train)

In [ ]:
future_df = m.make_future_dataframe(periods=13, freq='W')
future_df['ds'] = pd.to_datetime(future_df['ds'])
future_df = future_df.merge(shelter_df[['ds', 'Temperature']], on='ds', how='left')
future_df

In [ ]:
predictions = m.predict(future_df)
predictions = predictions.set_index('ds').join(shelter_df.set_index('ds')['y'])

rmse = root_mean_squared_error(predictions['yhat'], predictions['y'])
mae = mean_absolute_error(predictions['yhat'], predictions['y'])
[rmse, mae]

In [ ]:
plot_plotly(m, predictions.reset_index())

In [ ]:
plot_components_plotly(m, predictions.reset_index())

### Parameter Tuning

In [24]:
param_grid = {'changepoint_prior_scale':[0.01, 0.1, 0.5],
              'seasonality_prior_scale': [0.1, 1, 10],
              'holidays_prior_scale': [0.1, 1, 10],
              'seasonality_mode': ['additive', 'multiplicative']}

all_params = list(ParameterGrid(param_grid))

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

metrics_list = []
for params in all_params:

    m = Prophet(holidays=holidays, **params)
    m.add_regressor('Temperature')

    m.fit(df_train)

    df_cv = cross_validation(model=m,
                        period='42 days',
                        initial='1500 days',
                        horizon='91 days',
                        parallel='processes')
    
    metrics = {'rmse': performance_metrics(df_cv, rolling_window=1)['rmse'].mean(), 
               'mape': performance_metrics(df_cv, rolling_window=1)['mape'].mean()}
    
    metrics_list.append(metrics)

In [ ]:
params_df = pd.DataFrame(all_params)
params_df[['rmse', 'mape']] = pd.DataFrame(metrics_list)
params_df.query('rmse == rmse.min()')

In [ ]:
best_params = params_df.query('rmse == rmse.min()').transpose()[:-2].to_dict()[36]
best_params

In [ ]:
m = Prophet(holidays=holidays, **best_params)
m.add_regressor('Temperature')

m.fit(df_train)

df_cv = cross_validation(model=m,
                        period='42 days',
                        initial='1500 days',
                        horizon='91 days',
                        parallel='processes')

In [58]:
predictions = m.predict(future_df)
predictions = predictions.set_index('ds').join(shelter_df.set_index('ds')['y'])


In [ ]:
plot_plotly(m, predictions.reset_index())

In [ ]:
plot_components_plotly(m, predictions.reset_index())